In [2]:
!pip install --upgrade --no-cache-dir gdown >/dev/null
!gdown 1-OCBGBtKoY_PadKHcXDyWxHQ2BS8Nulo

Downloading...
From: https://drive.google.com/uc?id=1-OCBGBtKoY_PadKHcXDyWxHQ2BS8Nulo
To: /content/bigdata_hw1_files.zip
100% 38.9M/38.9M [00:00<00:00, 45.2MB/s]


In [3]:
!unzip bigdata_hw1_files.zip

Archive:  bigdata_hw1_files.zip
   creating: hw1-files/
   creating: hw1-files/q3/
  inflating: hw1-files/q3/patches.csv  
  inflating: hw1-files/q3/lsh.py     
   creating: hw1-files/q1/
  inflating: hw1-files/q1/dataset1.txt  
   creating: hw1-files/q1/.ipynb_checkpoints/
   creating: hw1-files/q2/
  inflating: hw1-files/q2/games_library.txt  
   creating: hw1-files/.ipynb_checkpoints/


In [4]:
from itertools import combinations
from tabulate import tabulate

S = 100
R = 5
INPUT_FILE = 'hw1-files/q2/games_library.txt'

In [5]:
# Read input file and fill baskets
baskets = []
with open(INPUT_FILE) as f:
    baskets = [frozenset(l.split()) for l in f.readlines()]

In [6]:
def freq_itemsets(k, c1=None, cp=None):
    """
    Generate frequent itemsets of level k using
    frequent items and previous level frequent itemsets.

    Args:
        k (int): Level number.
        c1 (dict, optional): Frequent Items. (from level 1)
        cp (dict, optional): Frequent Itemsets of level k-1 (previous level).

    Returns:
        dict: Frequent Itemsets of level k. (Itemset as key and support as value) 
    """

    c = {}

    # Pass 1
    # Read baskets and count in main memory the occurrences of each individual item.
    # Remove non-frequent items.
    if k == 1:
        for basket in baskets:
            for item in basket:
                c[item] = c.get(item, 0) + 1
        c = {item: c[item] for item in c if c[item] >= S}

    # Pass k
    # Generate sets with k items through combinations of frequent items.
    # Check whether all subsets of k-1 size are frequent.
    # Read baskets and count the occurrences of each itemset of k size.
    # Remove non-frequent itemsets.
    else:
        for itemset in combinations(c1, k):
            if k == 2 or all([frozenset(pset) in cp for pset in combinations(itemset, k - 1)]):
                itemset = frozenset(itemset)
                for basket in baskets:
                    if itemset.issubset(basket):
                        c[itemset] = c.get(itemset, 0) + 1
        c = {itemset: c[itemset] for itemset in c if c[itemset] >= S}

    return c

In [7]:
def gen_rules(k, ck, cp):
    """
    Generate association rules with their confidence
    using frequent itemsets of level k and level k-1.

    Args:
        k (int): Level number.
        ck (dict): Frequent Itemsets of level k.
        cp (dict): Frequent Itemsets of level k-1 (previous level).

    Returns:
        list: 2-dim list of top R rules.
    """
    rules = []
    for itemset in ck:
        sup_lr = ck[itemset]
        for left_set in combinations(itemset, k - 1):
            left_set = frozenset(left_set)
            right = itemset.difference(left_set)
            sup_l = cp[left_set if k > 2 else list(left_set)[0]]
            conf = sup_lr / sup_l
            left = list(left_set)
            left.sort()
            rules.append([' , '.join(left), *right, conf])
    rules = sorted(rules, key=lambda x: x[1])
    rules = sorted(rules, key=lambda x: x[2], reverse=True)
    return rules[:R]

In [13]:
def print_table(rules):
    """
    Print the rules in a pretty table.

    Args:
        rules (list): 2-dim list of rules.
    """
    header = ['Left Side of Rule', 'Right Side of Rule', 'Confidence']
    rows = [header] + rules
    table = tabulate(rows, headers='firstrow', tablefmt='fancy_grid')
    print(table)

In [10]:
c1 = freq_itemsets(k=1)
c2 = freq_itemsets(k=2, c1=c1, cp=c1)
c3 = freq_itemsets(k=3, c1=c1, cp=c2)

In [14]:
print_table(gen_rules(k=2, ck=c2, cp=c1))

╒══════════════════════╤════════════════════════╤══════════════╕
│ Left Side of Rule    │ Right Side of Rule     │   Confidence │
╞══════════════════════╪════════════════════════╪══════════════╡
│ Dying_Light_2        │ The_Last_of_Us_Part_II │     0.915021 │
├──────────────────────┼────────────────────────┼──────────────┤
│ ARK:Survival_Evolved │ The_Last_of_Us_Part_II │     0.913667 │
├──────────────────────┼────────────────────────┼──────────────┤
│ ARK:Survival_Evolved │ GTA_V                  │     0.913616 │
├──────────────────────┼────────────────────────┼──────────────┤
│ ARK:Survival_Evolved │ UNCHARTED_4            │     0.912544 │
├──────────────────────┼────────────────────────┼──────────────┤
│ Ghost_of_Tsushima    │ UNCHARTED_4            │     0.911961 │
╘══════════════════════╧════════════════════════╧══════════════╛


In [15]:
print_table(gen_rules(k=3, ck=c3, cp=c2))

╒══════════════════════════════════════════════╤════════════════════════╤══════════════╕
│ Left Side of Rule                            │ Right Side of Rule     │   Confidence │
╞══════════════════════════════════════════════╪════════════════════════╪══════════════╡
│ Assassin's_Creed_Odyssey , DAYS_GONE         │ The_Last_of_Us_Part_II │     0.963583 │
├──────────────────────────────────────────────┼────────────────────────┼──────────────┤
│ A_Way_Out , Ghost_of_Tsushima                │ UNCHARTED_4            │     0.963477 │
├──────────────────────────────────────────────┼────────────────────────┼──────────────┤
│ Far_Cry_6 , Ghost_of_Tsushima                │ GTA_V                  │     0.963398 │
├──────────────────────────────────────────────┼────────────────────────┼──────────────┤
│ ARK:Survival_Evolved , Red_Dead_Redemption_2 │ UNCHARTED_4            │     0.963388 │
├──────────────────────────────────────────────┼────────────────────────┼──────────────┤
│ Ghost_of_Tsushima ,